In [1]:
# Import dependencies
import pickle
import os
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import numpy as np
import requests
import io

In [2]:
# THIS IS ONLY TO BE USED IF THE GOOGLE DRIVE IS NOT MOUNTED ALREADY
# This will prompt for authorization.
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
# Load the images from the vehicle pickle file
with open('/content/drive/MyDrive/Colab Notebooks/Final Project/preprocessed_vehicles.pkl', 'rb') as file:
    preprocessed_vehicle_imgs = pickle.load(file)

preprocessed_vehicle_imgs[0]

array([[[0.75686276, 0.17254902, 0.2       ],
        [0.7529412 , 0.17254902, 0.19607843],
        [0.7411765 , 0.16862746, 0.1882353 ],
        ...,
        [0.11764706, 0.02352941, 0.01568628],
        [0.11764706, 0.02352941, 0.01568628],
        [0.11764706, 0.02352941, 0.01568628]],

       [[0.7294118 , 0.14901961, 0.1764706 ],
        [0.7254902 , 0.14901961, 0.17254902],
        [0.7176471 , 0.14509805, 0.16470589],
        ...,
        [0.11764706, 0.02352941, 0.01568628],
        [0.11764706, 0.02352941, 0.01568628],
        [0.11764706, 0.02352941, 0.01568628]],

       [[0.6745098 , 0.11372549, 0.13725491],
        [0.6745098 , 0.11372549, 0.13333334],
        [0.67058825, 0.10980392, 0.12941177],
        ...,
        [0.11764706, 0.02745098, 0.01960784],
        [0.11764706, 0.02745098, 0.01960784],
        [0.11764706, 0.02745098, 0.01960784]],

       ...,

       [[0.72156864, 0.42745098, 0.3137255 ],
        [0.72156864, 0.42745098, 0.3137255 ],
        [0.72156864, 0

In [4]:
# Load from the vehicles_y pickle file
with open('/content/drive/MyDrive/Colab Notebooks/Final Project/vehicles_y.pkl', 'rb') as file:
    vehicles_y = pickle.load(file)

vehicles_y[0]

'car'

In [5]:
# Continue loading
X = preprocessed_vehicle_imgs
y = vehicles_y

print(X[0])
print(y.head(5))

[[[0.75686276 0.17254902 0.2       ]
  [0.7529412  0.17254902 0.19607843]
  [0.7411765  0.16862746 0.1882353 ]
  ...
  [0.11764706 0.02352941 0.01568628]
  [0.11764706 0.02352941 0.01568628]
  [0.11764706 0.02352941 0.01568628]]

 [[0.7294118  0.14901961 0.1764706 ]
  [0.7254902  0.14901961 0.17254902]
  [0.7176471  0.14509805 0.16470589]
  ...
  [0.11764706 0.02352941 0.01568628]
  [0.11764706 0.02352941 0.01568628]
  [0.11764706 0.02352941 0.01568628]]

 [[0.6745098  0.11372549 0.13725491]
  [0.6745098  0.11372549 0.13333334]
  [0.67058825 0.10980392 0.12941177]
  ...
  [0.11764706 0.02745098 0.01960784]
  [0.11764706 0.02745098 0.01960784]
  [0.11764706 0.02745098 0.01960784]]

 ...

 [[0.72156864 0.42745098 0.3137255 ]
  [0.72156864 0.42745098 0.3137255 ]
  [0.72156864 0.42745098 0.3137255 ]
  ...
  [0.74509805 0.6117647  0.5764706 ]
  [0.7490196  0.6156863  0.5803922 ]
  [0.7490196  0.6156863  0.5803922 ]]

 [[0.7176471  0.42352942 0.30980393]
  [0.7176471  0.42352942 0.30980393]


In [6]:
# Label encode the y data, fit, and transform
y_encoder = LabelEncoder().fit(y)
y = y_encoder.transform(y)

In [7]:
# Double check the shapes of your images
for i, img in enumerate(X):
    print(f"Image {i} shape: {img.shape}")

Image 0 shape: (250, 250, 3)
Image 1 shape: (250, 250, 3)
Image 2 shape: (250, 250)
Image 3 shape: (250, 250, 3)
Image 4 shape: (250, 250, 3)
Image 5 shape: (250, 250, 3)
Image 6 shape: (250, 250, 3)
Image 7 shape: (250, 250, 3)
Image 8 shape: (250, 250, 3)
Image 9 shape: (250, 250, 3)
Image 10 shape: (250, 250, 3)
Image 11 shape: (250, 250, 3)
Image 12 shape: (250, 250, 3)
Image 13 shape: (250, 250, 3)
Image 14 shape: (250, 250, 3)
Image 15 shape: (250, 250, 3)
Image 16 shape: (250, 250, 3)
Image 17 shape: (250, 250, 3)
Image 18 shape: (250, 250, 3)
Image 19 shape: (250, 250, 3)
Image 20 shape: (250, 250, 3)
Image 21 shape: (250, 250, 3)
Image 22 shape: (250, 250, 3)
Image 23 shape: (250, 250, 3)
Image 24 shape: (250, 250, 3)
Image 25 shape: (250, 250, 3)
Image 26 shape: (250, 250, 3)
Image 27 shape: (250, 250, 3)
Image 28 shape: (250, 250, 3)
Image 29 shape: (250, 250, 3)
Image 30 shape: (250, 250, 3)
Image 31 shape: (250, 250, 3)
Image 32 shape: (250, 250, 3)
Image 33 shape: (250, 2

In [8]:
# Ensure all images have the same number of channels for RGB

def ensure_rgb(img):
    if len(img.shape) == 2:  # Grayscale image
        return np.stack([img, img, img], axis=-1)
    elif img.shape[2] == 4:  # RGBA image
        return img[:,:,:3]
    elif img.shape[2] == 3:  # Already RGB
        return img
    else:
        raise ValueError(f"Unexpected image shape: {img.shape}")

X = [ensure_rgb(img) for img in X]

In [9]:
# CREATE A NUMPY ARRAY
X = np.array(X)

In [10]:
# Split the training dataset into training and validation sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [11]:
# Define the input shape
input_shape = (250, 250, 3)

In [12]:
# Define a CNN model; make sure to use the correct values for input shape!
# HINT: Check the preprocessing activity to see the dimensions we used.

model = models.Sequential([
    layers.Input(shape=input_shape),
    layers.Conv2D(32, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(2, activation='sigmoid')
])

In [13]:
# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [14]:
# Train the model
batch_size = 32
epochs = 2
history = model.fit(
    X_train, y_train,
    validation_data=(X_test, y_test),
    epochs=epochs
)

Epoch 1/2
100/100 ━━━━━━━━━━━━━━━━━━━━ 8s 51ms/step - accuracy: 0.5934 - loss: 7.3303 - val_accuracy: 0.8500 - val_loss: 0.3615
Epoch 2/2
100/100 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - accuracy: 0.9215 - loss: 0.2078 - val_accuracy: 0.8800 - val_loss: 0.3150
